In [129]:
# # Installing necessary dependencies
import sys
import os
# !conda install --yes --prefix {sys.prefix} PIL
# # !{sys.executable} -m pip install ____

# !pip install openpyxl
# !{sys.executable} -m pip install pytesseract
# !{sys.executable} -m pip install pillow
# !{sys.executable} -m pip install python-poppler
# !{sys.executable} -m pip install opencv-python
# !{sys.executable} -m pip install --upgrade pip
# !conda install -c conda-forge poppler

# !{sys.executable} -m pip install scipy
# !{sys.executable} -m pip install sklearn
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install nltk
# !{sys.executable} -m pip install pandas


In [80]:
# Checking which environment we are working in
# !conda info --envs # check which env we are in
# !conda activate HCP-env # Activate the right environment

In [130]:
# Below, I try to do some text processing and tokenization to see word frequencies

import codecs
import re
import copy
import collections

import numpy as np
import pandas as pd
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import WordPunctTokenizer
from __future__ import division
# import matplotlib
import json


In [131]:
nltk.download('stopwords') # downloads all stopwords to avoid

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\15713\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [132]:
from nltk.corpus import stopwords 

In [133]:
# getting a list of unecessary words like 'the', 'or',... to avoid
avoid = stopwords.words('english')
additional_words = ['edu', 'transcript', 'hour', 'hours', 'program', 'experience', 'work', 'must', 'provide']
avoid = avoid + additional_words# adding our own word to the list


In [134]:
nltk.download('punkt') # downloads the tokenizer we need

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\15713\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [135]:
# Function to tokenize a text file into sentences or phrases
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def get_sentence_counter(path=None, text=None):
    if path:
        path = os.getcwd()
        for file in os.listdir():
            if file.endswith('.txt'):
                with codecs.open(file, 'r') as f: 
                    text = f.read()
    else:
        text = text
    raw_sentences = tokenizer.tokenize(text)
    sentences = []
    for raw_sentence in raw_sentences:
        tmp = raw_sentence.split('\n')
        for x in tmp:
            sentences.append(x)

    return sentences, len(sentences)

In [136]:
# Function to tokenize a text file into words
def remove_stopwords(text, avoid):
#     tokens = WordPunctTokenizer().tokenize(PorterStemmer().stem(text))
    tokens = WordPunctTokenizer().tokenize(text)
    tokens = list(map(lambda x: x.lower(), tokens)) # converting everything to lower-case
    
    # cleaning up tokens by getting rid of stopwords and finding word characters
    tokens = [token for token in tokens if token not in avoid]
#     Puncs = [token for token in tokens if re.match(re.compile("\W"), token)] # finding all punctuations
#     Nums = [token for token in tokens if re.match(re.compile("\d"), token)] # finding all digit characters
    # adding punctuations and digits to the list of avoided characters/words
#     avoid = avoid + Puncs + Nums
#     tokens = [token for token in tokens if re.match(word_pattern, token) and token not in avoid] 
    # collections.counter() returns the tokens and their frequency as a dictionary ({freq: token})
    sentence = ''
    for token in tokens:
        sentence = sentence + token + ' '
    return sentence

In [137]:
text = "initialization:and add Tesseract OCR to the environment Add the following to the code"
p_text = remove_stopwords(text, avoid)
p_text

'initialization : add tesseract ocr environment add following code '

In [ ]:
################################################################################
################################################################################
################################################################################
# INDEED

In [138]:
# Working with 'Master_Indeed_Dataset'
path = r'C:\Users\15713\Desktop\Datasets\csv_xlsx\Master_Indeed_Dataset.xlsx'
df = pd.read_excel(path)

In [31]:
df.head(6)

,Title,Company,Location,Links,Salary,Description
0,JUNIOR PENETRATION TESTER,Elevate,Remote,https://indeed.com/viewjob?jk=0a12694d75919f0c,$35 - $50 an hour,JUNIOR PENETRATION TESTER\nwww.elevateconsult....
1,Junior Penetration Tester USA Remote,BreachLock,Floridaâ€¢Remote,https://indeed.com/viewjob?jk=1c98609eab9789cc,None,The Penetration Testing professional should ha...
2,Red Team - Penetration Tester,"Simplicity, LLC.",Remote,https://indeed.com/viewjob?jk=1256f1d0f5d5af0c,"$75,000 - $90,000 a year",Who are we?\nBreachLock is a security startup ...
3,Penetration Tester,Creative Visions,"Washington, DCâ€¢Remote",https://indeed.com/viewjob?jk=c52fc17b016d03d5,None,"Fort Mill, SC, Carrollton TX â€“ Primarily Rem..."
4,Penetration Tester,TechTrueUP,Remote,https://indeed.com/viewjob?jk=d344378f0a869289,$45 - $50 an hour,Summary:\nThe Penetration Tester role will wor...
5,Penetration Tester (remote),Fast AF,"San Francisco, CAâ€¢Remote",https://indeed.com/viewjob?jk=e52b91673b7586f7,None,Job Details:\nIn this vital role you will ensu...


In [139]:
# Creating a dataframe with title as index and description as an attribute
title, description = df['Title'].to_list(), df['Description'].to_list()
indeed_df = pd.DataFrame(data={"Title": title,
                               "Description": description} 
                         )
# removing rows with 'None' for their Job-Title
indeed_df = indeed_df[indeed_df.Title != 'None']

In [33]:
indeed_df.head(3)

,Title,Description
0,JUNIOR PENETRATION TESTER,JUNIOR PENETRATION TESTER\nwww.elevateconsult....
1,Junior Penetration Tester USA Remote,The Penetration Testing professional should ha...
2,Red Team - Penetration Tester,Who are we?\nBreachLock is a security startup ...


In [140]:
# Converting the above dataframe into dictionary for iteration {title: description}
indeed = {}
for (t, d) in zip(title, description): # loops through the list title and description
    if t in indeed and t!='None': # if the job title already exists, we lump the descriptions together
        indeed[t]+=d
    else:
        indeed[t] = d # sets the title as key and the description as the value


In [141]:
# iterating through the Job's to tokenize into sentences/phrases
indeed_tokenized_dict = {}
for key in indeed:
    if key != 'None':
        text = indeed[key]
        # Tokenize into sentence or phrase
        sentence_count, s1 = get_sentence_counter(text=text)
        indeed_tokenized_dict[key] = sentence_count


In [142]:
# Putting in dataframe with keys/job titles as titles and the values as the tokenized sentences just for visual purposes
df_concat = pd.DataFrame()
for k,v in indeed_tokenized_dict.items():
    if k != 'None':
        df = pd.DataFrame({k: v})
        df_concat = pd.concat([df_concat, df], axis=1)

df_concat.T.head(30)

,0,1,2,3,4,5,6,7,8,9,...,1445,1446,1447,1448,1449,1450,1451,1452,1453,1454
JUNIOR PENETRATION TESTER,JUNIOR PENETRATION TESTER,www.elevateconsult.com,ABOUT ELEVATE,Elevate is a premier consulting firm specializ...,"Our team consists of self-managed, high-calibe...","We value exceptional client service, solving a...","We are looking for a motivated, experienced an...",ABOUT THE POSITION,This is a contract to-hire-position which star...,Elevate is looking for a Junior Penetration Te...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Junior Penetration Tester USA Remote,The Penetration Testing professional should ha...,The consultant would perform various penetrati...,The assessments that would be delivered would ...,For the remote assessments the consultant woul...,The consultant should be comfortable identifyi...,The consultant should have a basic understandi...,Must Haves:,"Background in Enterprise network, application,...","1-3 years experience performing application, m...",3-5 years of experience in vulnerability asses...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Red Team - Penetration Tester,Who are we?,BreachLock is a security startup that offers a...,The BreachLock platform leverages both human-p...,BreachLocksâ€™s modern SaaS-based approach red...,"As a result, CIOâ€™s and CISOâ€™s get a single...",The BreachLock platform facilitates collaborat...,Some of our achievements include:,One of the fastest-growing SaaS companies in C...,Cyber Security Innovator for Analysis and Test...,Top 10 Vulnerability Management Solution for 2...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Penetration Tester,"Fort Mill, SC, Carrollton TX â€“ Primarily Rem...",This is a fast track hire (6mth to hire).,See major requirements to consider below:,Â· Position will require 70% Business / 30% Te...,Â· Demonstrates the ability to manage various ...,Â· Provide technical/management leadership on ...,Â· Minimum experience(actual work experience) ...,Â· Ability define security scope while alignin...,Â·,POSITION SUMMARY:,...,Lead engagements and assist less experienced s...,BASIC QUALIFICATIONS,"Bachelorâ€™s degree in cybersecurity, computer...",At least three years of experience in a securi...,Strong attention to detail and problem solving...,As a seven-time Star Tribune Top Workplace we ...,When working at HelpSystems you will have the ...,Check out our website to learn more about Help...,We encourage you to check out our Glassdoor pa...,"As an EEO/Affirmative Action Employer, all qua..."
Penetration Tester (remote),Job Details:,In this vital role you will ensure that flaws ...,"Define, lead and execute red team/blue team ex...",Management of technologies and processes relat...,Develop or participate in the development of b...,"Advise project teams, application owners, and ...",Perform security design consulting in support ...,The creative professional we seek is a self-dr...,Excellent analytical and troubleshooting skills,"Must be team-oriented, placing priority on the...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
(Remote) Penetration Tester,Our cybersecurity company is seeking a passion...,You will not get bored in our fast-paced work ...,Successful team members strive for excellence ...,Our office is located in the heart of downtown...,To learn more about how you may be able to fit...,If you think you are our ideal candidate apply...,Essential Functions:,Conducting tests on applications and networks:...,"This includes penetration tests, red team test...",Social engineering tests: Conduct remote and o...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cybersecurity Penetration Tester-Web Applications,"Creative Visions is seeking a motivated, caree...",The Penetration Tester will be responsible for...,The ideal candidate will have a basic understa...,"Testing includes web application assessments, ...",Develop and document security evaluation test ...,Provide technical expertise and guidance in de...,Responsible for aligning industry best securit...,Maint

In [338]:
# shows the description
# indeed['SOC Analyst']

In [337]:
# shows the sentences in the description tokenized
# indeed_tokenized_dict['SOC Analyst']

In [143]:
# Here I will try to create 'bag of words' that could be used to identify if a sentence is refering to 
# ability, skill, knowledge or other
bag_A = [' able ', ' ability', 'Ability ', ' abilities', 'perform']
bag_S = ['skill', 'Skill',]
bag_K = ['know', 'known', 'knowledge', 'Knowledge']
bag_C = ['certification', 'Certification']

# Helper function to check whether the flag words are present in the sentence
def check_word(word_list,sentence):
    for x in word_list:
        return True if x in sentence else False

In [144]:
# Now I will sort out the sentences into Ability, skill, knowledge or Certs and save it as a nested dict
Master_indeed_dict = {}
pattern = '[\s|^|^\s][A-Z(][A-Z]+[\s|)|,]'
I_dict = {}
for k in indeed_tokenized_dict.keys():
    K,S,A,C = [],[],[],[]
    for sentence in indeed_tokenized_dict[k]:
        if check_word(bag_K,sentence):
            sentence = remove_stopwords(sentence, avoid) # Removing stopwords and reconstructing sentence
            K.append(sentence)
        if check_word(bag_S,sentence):
            sentence = remove_stopwords(sentence, avoid) # Removing stopwords and reconstructing sentence
            S.append(sentence)
        if check_word(bag_A,sentence):
            sentence = remove_stopwords(sentence, avoid) # Removing stopwords and reconstructing sentence
            A.append(sentence)
        # Here, I tried to use regex to find patterns matching a valid Certificate by first
        # finding a sentence that contains the flag words
        if check_word(bag_C, sentence):
            print(f'\nsentence: {sentence}') #printing the sentence to be checked for valid certs
            match = re.findall(pattern, sentence) #checking words matching our pattern
            print('found certs...')
            for x in match:
                x = re.sub('\W', '', x) #removes any non-word characters (space,symbols)
                C.append(x) #adding cleaned up string to the certs list
                print(f'-{x}')
  
    Master_indeed_dict[k] = {'K':K,
                             'S':S,
                             'A':A,
                             'C':list(set(C))
                            }
    #for visualization purose only:
#     I_dict[k] = K+S+A+C
    

    


sentence: Working towards or more security certifications: OSCP, LPT, CPT, CEH, CEPT, CMWAPT, CRTOP, Comp TIA PenTest+, GPEN
found certs...
-OSCP
-LPT
-CPT
-CEH
-CEPT
-CMWAPT
-CRTOP
-TIA

sentence: Have certifications like CEH or ISO 27001
found certs...
-CEH
-ISO

sentence: Â· Requires Security Certification(s) (i.e., Certified Information Systems Security Professional (CISSP), or Certified Information Security Manage (CISM) or Certified Ethical Hacker (CEH), Offensive Security Certified Professional (OSCP), eLearnSecurity Web application Penetration Tester (eWPT)or other equivalent recognized security certifications (OSCP & eWPT is major plus)
found certs...
-CISSP
-CISM
-CEH
-OSCP
-OSCP

sentence: Information Technology and Security certifications such as Security+, CEH, GIAC, OSCP, or similar preferred.
found certs...
-CEH
-GIAC
-OSCP

sentence: CEH, OSCP or similar certification
found certs...
-OSCP

sentence: Â· Offensive Security Certified Professional (OSCP) and/or GIAC certif


sentence: Applicant must hold one active relevant technical security certification prior to selection (e.g., CISSP, CISA, CISM).
found certs...
-CISSP
-CISA
-CISM

sentence: If you claim membership, license, certification, or credentials, you must submit a copy of said document in your application package.
found certs...

sentence: Relevant work experience in Information Security, audit, risk management, or internal controls, or related certifications like Security+, CISSP, etc.
found certs...
-CISSP

sentence: Hold or pursuing certifications in Security+, CISSP, CISA, CISM, or similar certifications.
found certs...
-CISSP
-CISA
-CISM

sentence: You will coordinate activities contributing to the certification and accreditation of automated information systems (AISs) throughout the command.
found certs...

sentence: Subject matter expert with IT infrastructure, performance management methods, COTS products and components, evaluating, implementing and disseminating IT security tools and

In [145]:
I_df =  pd.DataFrame.from_dict({(i,j): Master_indeed_dict[i][j] 
                           for i in Master_indeed_dict.keys() 
                           for j in Master_indeed_dict[i].keys()},
                       orient='index')

In [146]:
I_df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,29,30,31,32,33,34,35,36,37,38
"(JUNIOR PENETRATION TESTER, K)",junior penetration tester support assessments ...,demonstrated knowledge vulnerability penetrati...,ability utilize technical tools ( online other...,intermediate advance knowledge various penetra...,intermediate advance knowledge networking oper...,intermediate advance knowledge programming lan...,intermediate advanced knowledge following lang...,strong working knowledge network security tech...,value safety member community know weâ €™ toge...,None,...,None,None,None,None,None,None,None,None,None,None
"(JUNIOR PENETRATION TESTER, S)","team consists self - managed , high - caliber ...",elevate looking junior penetration tester ( â ...,seek individuals enjoy learning add skillset q...,transparent reporting include forecasted sched...,candidate excellent verbal written communicati...,excellent client service project management sk...,strong analytical communications ( written ora...,role requires blend cybersecurity communicatio...,strong problem - solving skills effectively re...,skilled explaining technical problems succinct...,...,None,None,None,None,None,None,None,None,None,None
"(JUNIOR PENETRATION TESTER, A)","ideal candidate able think creatively , teams ...","able understand client needs , seek clarificat...",candidate excellent verbal written communicati...,"strong , confident , exacting writer speaker ,...",None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
"(JUNIOR PENETRATION TESTER, C)",CMWAPT,CPT,CRTOP,TIA,CEH,CEPT,LPT,OSCP,None,None,...,None,None,None,None,None,None,None,None,None,None
"(Junior Penetration Tester USA Remote, K)",ability acquire new knowledge skills quickly,coworkers clients â € œknow us resultsâ € â €...,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [174]:
# Master_indeed_dict['JUNIOR PENETRATION TESTER']['K']

In [175]:
# Master_indeed_dict['JUNIOR PENETRATION TESTER']['S']

In [ ]:
# Master_indeed_dict['JUNIOR PENETRATION TESTER']['A']

In [147]:
# Master_indeed_dict.keys()
Master_indeed_dict['Cybersecurity Specialist']['C']

['DAWIA', 'AFMAN', 'VOW', 'IAW', 'VEOA', 'VRA', 'DAU', 'II', 'VA', 'IT', 'EOD']

In [148]:
Master_indeed_dict['JUNIOR PENETRATION TESTER']['C']

['CMWAPT', 'CPT', 'CRTOP', 'TIA', 'CEH', 'CEPT', 'LPT', 'OSCP']

In [149]:
# Exporting dictionary as a json file for use during sentence vectorization
data = Master_indeed_dict
with open('Master_indeed_dict.json', 'w') as f:
    json.dump(data, f)

In [ ]:
################################################################################
################################################################################
################################################################################
# SYLLABI

In [150]:
#working with the syllabi data from NICCS
path = r'C:\Users\15713\Desktop\Datasets\csv_xlsx\syllabi_niccs(final).csv'
NICCS = pd.read_csv(path, encoding='cp1252')

In [43]:
NICCS.tail(5)

,College,City,State,Deg,AreaofStudy,Classroom,Online,CourseTitle,Description,LearningObjectives,FrameworkConnections,ConnectwithSpecialtyAreas,WorkRole,WorkRoleID,WorkRoleDesc,Abilities,Knowledge,Skills,Link
191,Florida A&M University,Tallahasse,FL,BS,Information Technology,1,0,Digital Forensics,Technical issues in acquiring computer evidenc...,Concepts in digital forensics including invest...,Investigate,Digital Forensics,Cyber Security Forensics Analyst,IN-FOR-002,Analyzes digital evidence and investigates com...,A0005: Ability to decrypt digital data collect...,K0001: Knowledge of computer networking concep...,"S0032: Skill in developing, testing, and imple...",https://niccs.cisa.gov/training/search/florida...
192,Florida A&M University,Tallahasse,FL,BS,Information Technology,1,0,Digital Forensics,Technical issues in acquiring computer evidenc...,Concepts in digital forensics including invest...,Investigate,Cyber Investigation,Cyber Crime Investigator,IN-INV-001,"Identifies, collects, examines, and preserves ...",A0174: Ability to Find and navigate the dark w...,K0001: Knowledge of computer networking concep...,S0047: Skill in preserving evidence integrity ...,https://niccs.cisa.gov/training/search/florida...
193,Florida A&M University,Tallahasse,FL,BS,Information Technology,1,0,Network Security and Cryptography,The goal of this course is to cover the theore...,Demonstrate an understanding of the theoretica...,Operate and Maintain; Protect and Defend,Cyber Defense Analysis,Cyber Defense Analyst,PR-CDA-001,Uses data collected from a variety of defense ...,A0010: Ability to analyze malware; A0015: Abil...,K0001: Knowledge of computer networking concep...,S0020: Skill in developing and deploying signa...,https://niccs.cisa.gov/training/search/florida...
194,Florida A&M University,Tallahasse,FL,BS,Information Technology,1,0,Network Security and Cryptography,The goal of this course is to cover the theore...,Demonstrate an understanding of the theoretica...,Operate and Maintain; Protect and Defend,Cyber Defense Infrastructure Support,Cyber Defense Infrastructure Support Specialist,PR-INF-001,"Test, deploys, maintains, and administers the ...",A0123: Ability to apply cybersecurity and priv...,K0001: Knowledge of computer networking concep...,S0007: Skill in applying host/network access c...,https://niccs.cisa.gov/training/search/florida...
195,Florida A&M University,Tallahasse,FL,BS,Information Technology,1,0,Network Security and Cryptography,The goal of this course is to cover the theore...,Demonstrate an understanding of the theoretica...,Operate and Maintain; Protect and Defend,Network Services,Network Operations Specialist,OM-NET-001,"Plans, implement, operates netwrok services/sy...",A0052: Ability to operate network equipment in...,K0001: Knowledge of computer networking concep...,S0004: Skill in analyzing network traffic capa...,https://niccs.cisa.gov/training/search/florida...


In [151]:
# Converting Columns needed to list for easier itteration
CourseTitle = NICCS['CourseTitle'].to_list()
WorkRole = NICCS['WorkRole'].to_list()
WorkRoleID = NICCS['WorkRoleID'].to_list()
Tasks = NICCS['WorkRoleDesc'].to_list()
Knowledge = NICCS['Knowledge'].to_list()
Skill = NICCS['Skills'].to_list()
Ability = NICCS['Abilities'].to_list()

In [152]:
# Saving each KSA under each work role as a nested dictionary
Syllabi_dict = {}
niccs_dict = {}
pattern = '[S|A|K][0-9]+[\S][\s]'
for i,WR in enumerate(WorkRole):
    K,S,A =[], [], []
    if WR not in Syllabi_dict.keys(): # Avoiding duplicates
        # splitting to get each KSA & IDs, while also removing the A####,K####, S#### tags
        k = re.sub(pattern, '', Knowledge[i]).split(";") 
        s = re.sub(pattern, '', Skill[i]).split(";")
        a = re.sub(pattern, '', Ability[i]).split(";")
        for sentence in k:
#             print(sentence)
            # Removing stopwords and reconstructing sentence
            K.append(remove_stopwords(sentence, avoid))
#             print('#######')
        for sentence in s:
            # Removing stopwords and reconstructing sentence
            S.append(remove_stopwords(sentence, avoid))
        for sentence in a:
            # Removing stopwords and reconstructing sentence
            A.append(remove_stopwords(sentence, avoid))
        Syllabi_dict[WR] = {'ID': WorkRoleID[i], 
                             'K': K, 
                             'S': S, 
                             'A': A} #Saving as a nested dict
        # for visualization purposes only:
        niccs_dict[WR] = [WorkRoleID[i]] + K + S + A

In [153]:
niccs_concat = pd.DataFrame()
for k,v in niccs_dict.items():
    if k != 'None':
        df = pd.DataFrame({k: v})
        niccs_concat = pd.concat([niccs_concat, df], axis=1)

niccs_concat.T.tail(30)

,0,1,2,3,4,5,6,7,8,9,...,133,134,135,136,137,138,139,140,141,142
Target Developer,AN-TGT-001,knowledge computer networking concepts protoco...,"knowledge risk management processes ( e . g .,...","knowledge laws , regulations , policies , ethi...",knowledge cybersecurity privacy principles,knowledge cyber threats vulnerabilities,knowledge specific operational impacts cyberse...,knowledge human - computer interaction princip...,knowledge network traffic analysis methods,"knowledge concepts , terminology , operations ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cyber Security Forensics Analyst,IN-FOR-002,knowledge computer networking concepts protoco...,"knowledge risk management processes ( e . g .,...","knowledge laws , regulations , policies , ethi...",knowledge cybersecurity privacy principles,knowledge cyber threats vulnerabilities,knowledge specific operational impacts cyberse...,knowledge encryption algorithms,knowledge data backup recovery,knowledge incident response handling methodolo...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cyber Crime Investigator,IN-INV-001,knowledge computer networking concepts protoco...,"knowledge risk management processes ( e . g .,...","knowledge laws , regulations , policies , ethi...",knowledge cybersecurity privacy principles,knowledge cyber threats vulnerabilities,knowledge specific operational impacts cyberse...,knowledge intrusion detection methodologies te...,knowledge system application security threats ...,"knowledge insider threat investigations , repo...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cyber Defense Infrastructure Support Specialist,PR-INF-001,knowledge computer networking concepts protoco...,"knowledge risk management processes ( e . g .,...","knowledge laws , regulations , policies , ethi...",knowledge cybersecurity privacy principles,knowledge cyber threats vulnerabilities,knowledge specific operational impacts cyberse...,knowledge data backup recovery,knowledge host / network access control mechan...,knowledge incident response handling methodolo...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cyber Defense Incident Responder,PR-CIR-001,knowledge computer networking concepts protoco...,"knowledge risk management processes ( e . g .,...","knowledge laws , regulations , policies , ethi...",knowledge cybersecurity privacy principles,knowledge cyber threats vulnerabilities,knowledge specific operational impacts cyberse...,knowledge data backup recovery,knowledge business continuity disaster recover...,knowledge host / network access control mechan...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Program Manager,OV-PMA-001,knowledge computer networking concepts protoco...,"knowledge risk management processes ( e . g .,...","knowledge laws , regulations , policies , ethi...",knowledge cybersecurity privacy principles,knowledge cyber threats vulnerabilities,knowledge specific operational impacts cyberse...,knowledge information technology ( ) architect...,knowledge risk management framework ( rmf ) re...,knowledge resource management principles techn...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Threat/Warning Analyst,AN-TWA-001,knowledge computer networking concepts protoco...,"knowledge risk management processes ( e . g .,...","knowledge laws , regulations , policies , ethi...",knowledge cybersecurity privacy principles,knowledge cyber threats vulnerabilities,knowledge specific operational impacts cyberse...,knowledge human - computer interaction princip...,knowledge network traffic analysis methods,"knowledge concepts , terminology , operations ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Vulnerability Assessment Analyst,PR-VAM-001,knowledge computer networking concepts protoco...,"knowledge risk management processes ( e . g .,...","knowledge laws , regulations , policies , ethi...",knowledge cybersecurity privacy principles,knowledge cyber threats vulnerabilities,knowledge specific operational impacts cyberse...,knowledge application vulnerabilities,knowledge 

In [154]:
syllabi_niccs_df =  pd.DataFrame.from_dict({(i,j): Syllabi_dict[i][j] 
                           for i in Syllabi_dict.keys() 
                           for j in Syllabi_dict[i].keys()},
                       orient='index')

In [53]:
syllabi_niccs_df.head(10)

,0
"(Network Operations Specialist, ID)",OM-NET-001
"(Network Operations Specialist, K)",[knowledge computer networking concepts protoc...
"(Network Operations Specialist, S)",[skill analyzing network traffic capacity perf...
"(Network Operations Specialist, A)",[ability operate network equipment including h...
"(Communications Security (COMSEC) Management, ID)",OV-MGT-002
"(Communications Security (COMSEC) Management, K)",[knowledge computer networking concepts protoc...
"(Communications Security (COMSEC) Management, S)",[skill determining security system work ( incl...
"(Communications Security (COMSEC) Management, A)",[ability recognize unique aspects communicatio...
"(Cyber Defense Analyst, ID)",PR-CDA-001
"(Cyber Defense Analyst, K)",[knowledge computer networking concepts protoc...


In [120]:
Syllabi_dict.keys()

dict_keys(['Network Operations Specialist', 'Communications Security (COMSEC) Management', 'Cyber Defense Analyst', 'Exploitation Analyst', 'Systems Security Analyst', 'Target Developer', 'Cyber Security Forensics Analyst', 'Cyber Crime Investigator', 'Cyber Defense Infrastructure Support Specialist', 'Cyber Defense Incident Responder', 'Program Manager', 'Threat/Warning Analyst', 'Vulnerability Assessment Analyst', 'Cyber Operator', 'System Testing and Evaluation Specialist', 'Cyber Intel Planner', 'Enterprise Architect', 'Authorizing Official/Designating Representative', 'Information Systems Security Manager', 'Cyber Workforce Developer and Manager', 'Systems Administrator', 'Systems Requirements Planner', 'Information Systems Security Developer', 'Cyber Ops Planner', 'Database Administrator', 'Law Enforcement/CounterIntelligence Forensics Analyst', 'Cyber Defense Forensics Analyst', 'Software Developer', 'Executive Cyber Leadership', 'All Source Collection Manager', 'Knowledge Manag

In [37]:
# Syllabi_dict['Network Operations Specialist']['K']

In [38]:
# Syllabi_dict['Network Operations Specialist']['S']

In [39]:
# Syllabi_dict['Network Operations Specialist']['A']

In [155]:
# Exporting dictionary as a json file for use during sentence vectorization
data = Syllabi_dict
with open('syllabi_NICCS.json', 'w') as f:
    json.dump(data, f)

In [ ]:
################################################################################
################################################################################
################################################################################
# Certs

In [156]:
#working with the Certificate data from NICCS
path = r'C:\Users\15713\Desktop\Datasets\csv_xlsx\cert_list.xlsx' 
Certs = pd.read_excel(path) 

In [56]:
Certs.head(5)

,Cert Name,Cost,Time (hrs),Avg Salary of Recipient,Number of Holders,Knowledge,Skills,Abilities
0,CISSP (Certified Information Systems Security ...,699,40-50,122138.0,92976.0,K0001: Knowledge of computer networking concep...,S0011: Skill in conducting information searche...,A0002: Ability to match the appropriate knowle...
1,CISA (Certified Information Systems Auditor),595,112,132278.0,150000.0,K0001: Knowledge of computer networking concep...,S0027: Skill in determining how a security sys...,A0162: Ability to recognize the unique aspects...
2,CISM (Certified Information Security Manager),760,40-50,148622.0,50000.0,K0001: Knowledge of computer networking concep...,S0027: Skill in determining how a security sys...,A0162: Ability to recognize the unique aspects...
3,Security+,370,45,72244.0,500000.0,K0001: Knowledge of computer networking concep...,S0020: Skill in developing and deploying signa...,A0010: Ability to analyze malware.\nA0013: Abi...
4,CEH (Certified Ethical Hacker),1199,70-75,104813.0,79617.0,K0001: Knowledge of computer networking concep...,S0005: Skill in applying and incorporating inf...,"A0008: Ability to apply the methods, standards..."


In [157]:
cert_name = Certs['Cert Name'].to_list()
c_K = Certs['Knowledge'].to_list()
c_S = Certs['Skills'].to_list()
c_A = Certs['Abilities '].to_list()

In [158]:
# Saving each KSA under each Cert as a nested dictionary
Cert_dict = {}
c_dict = {}
pattern = '[S|A|K][0-9]+[\S][\s]' 
for i,cert in enumerate(cert_name):
    K,S,A = [],[],[]
    if cert not in Cert_dict.keys(): # Avoiding duplicates
        k = 'None' if pd.isna(c_K[i]) else re.sub(pattern, '', c_K[i]).split("\n")
        s = 'None' if pd.isna(c_S[i]) else re.sub(pattern, '', c_S[i]).split("\n")
        a = 'None' if pd.isna(c_A[i]) else re.sub(pattern, '', c_A[i]).split("\n")
        
        for sentence in k:
            # Removing stopwords and reconstructing sentence
            K.append(remove_stopwords(sentence, avoid))
        for sentence in s:
            # Removing stopwords and reconstructing sentence
            S.append(remove_stopwords(sentence, avoid))
        for sentence in a:
            # Removing stopwords and reconstructing sentence
            A.append(remove_stopwords(sentence, avoid))
        
        Cert_dict[cert] = {'K': K, 
                           'S': S, 
                           'A': A} #Saving as a nested dict
        #for visualization purpose only:
        c_dict[cert] = K+S+A

In [159]:
c_dict = pd.DataFrame()
for k,v in Cert_dict.items():
    if k != 'None':
        df = pd.DataFrame({k: v})
        c_dict = pd.concat([c_dict, df], axis=1)

c_dict.T.head(5)

,A,K,S
CISSP (Certified Information Systems Security Professional),[ability match appropriate knowledge repositor...,[knowledge computer networking concepts protoc...,"[skill conducting information searches . , ski..."
CISA (Certified Information Systems Auditor),[ability recognize unique aspects communicatio...,[knowledge computer networking concepts protoc...,[skill determining security system ( including...
CISM (Certified Information Security Manager),[ability recognize unique aspects communicatio...,[knowledge computer networking concepts protoc...,[skill determining security system ( including...
Security+,"[ability analyze malware . , ability communica...",[knowledge computer networking concepts protoc...,"[skill developing deploying signatures . , ski..."
CEH (Certified Ethical Hacker),"[ability apply methods , standards , approache...",[knowledge computer networking concepts protoc...,[skill applying incorporating information tech...


In [160]:
cert_niccs_df =  pd.DataFrame.from_dict({(i,j): Cert_dict[i][j] 
                           for i in Cert_dict.keys() 
                           for j in Cert_dict[i].keys()},
                       orient='index')

In [161]:
cert_niccs_df.head(3)

,0,1,2,3,4,5,6,7,8,9,...,333,334,335,336,337,338,339,340,341,342
"(CISSP (Certified Information Systems Security Professional), K)",knowledge computer networking concepts protoco...,"knowledge risk management processes ( e . g .,...","knowledge laws , regulations , policies , ethi...",knowledge cybersecurity privacy principles .,knowledge cyber threats vulnerabilities .,knowledge specific operational impacts cyberse...,knowledge application vulnerabilities .,knowledge cyber defense vulnerability assessme...,knowledge cryptography cryptographic key manag...,knowledge organization ' enterprise informatio...,...,None,None,None,None,None,None,None,None,None,None
"(CISSP (Certified Information Systems Security Professional), S)",skill conducting information searches .,"skill conducting knowledge mapping ( e . g ., ...",skill creating policies reflect system securit...,"skill discerning protection needs ( . e ., sec...",skill measuring reporting intellectual capital .,skill using knowledge management technologies .,"skill administrative planning activities , inc...",skill preparing plans related correspondence .,skill communicating levels management includin...,skill anticipate new security threats .,...,None,None,None,None,None,None,None,None,None,None
"(CISSP (Certified Information Systems Security Professional), A)",ability match appropriate knowledge repository...,ability determine validity technology trend da...,ability assess forecast manpower requirements ...,"ability develop policy , plans , strategy comp...",ability leverage best practices lessons learne...,ability oversee development update life cycle ...,ability apply critical reading / thinking skil...,ability coordinate cyber operations organizati...,ability exercise judgment policies well - defi...,ability identify external partners common cybe...,...,None,None,None,None,None,None,None,None,None,None


In [44]:
# Cert_dict.keys()

In [45]:
# Cert_dict['CISA (Certified Information Systems Auditor)']['A']

In [162]:
# Exporting dictionary as a json file for use during sentence vectorization
data = Cert_dict
with open('Certs.json', 'w') as f:
    json.dump(data, f)

In [ ]:
############## HERE I JUST WANTED TO TRY TO GROUP SIMILAR JOB-TITLES(JOB POSTINGS) AND JOB-ROLES(SYLLABI) TOGETHER
############## USING "THE MOST COMMON WORD" TECHNIQUE

In [163]:

# function that returns the number of common words in two sentences

def common_words_count(s1, s2):
    count = 0
    # Convert sentence to all lower case and split them into words
    try:
        l1 = s1.lower().split(' ') if s1==s1 else None
        l2 = s2.lower().split(' ') if s2==s2 else None

        #Lemmetization if necessary (but for now, i will try it without)

        if l1 is not None:
            for w1 in l1:
                if l2 is not None and w1 in l2:
                    count+=1
    except:
        pass
    return count



In [164]:
#working with the Reference data from NICCS
path = r'C:\Users\15713\Desktop\Datasets\csv_xlsx\NIST_NICE_Reference_Data.xlsx' 
Reference = pd.read_excel(path, sheet_name='WorkRoles') 

In [169]:
Reference.head()

,WR_ID,WR
0,CO-CLO-001,All Source-Collection Manager
1,CO-CLO-002,All Source-Collection Requirements Manager
2,AN-ASA-001,All-Source Analyst
3,SP-RSK-001,Authorizing Official/Designating Representative
4,OV-MGT-002,Communications Security (COMSEC) Manager


In [170]:
WR = Reference['WR'].to_list()
WR_ID = Reference['WR_ID'].to_list()

In [171]:
print(WR)

['All Source-Collection Manager', 'All Source-Collection Requirements Manager', 'All-Source Analyst', 'Authorizing Official/Designating Representative', 'Communications Security (COMSEC) Manager', 'Cyber Crime Investigator', 'Cyber Defense Analyst', 'Cyber Defense Forensics Analyst', 'Cyber Defense Incident Responder', 'Cyber Defense Infrastructure Support Specialist', 'Cyber Instructional Curriculum Developer', 'Cyber Instructor', 'Cyber Intel Planner', 'Cyber Legal Advisor', 'Cyber Operator', 'Cyber Ops Planner', 'Cyber Policy and Strategy Planner', 'Cyber Workforce Developer and Manager', 'Data Analyst', 'Database Administrator', 'Enterprise Architect', 'Executive Cyber Leadership', 'Exploitation Analyst', 'Information Systems Security Developer', 'Information Systems Security Manager', 'IT Investment/Portfolio Manager', 'IT Program Auditor', 'IT Project Manager', 'Knowledge Manager', 'Law Enforcement/Counterintelligence Forensics Analyst', 'Mission Assessment Specialist', 'Multi-Di

In [172]:
min_count = 4

similar = {}
others = []
for K in Master_indeed_dict.keys():
    print('###############################################')
#     print(K)
#     print(sim)
    for k in WR:
        print(f'WR:{k}')
        sim = []
        if k==k and K==K: # making sure that we don't have a nan 
            K_words = K.lower().split(' ')
            k_words = k.lower().split(' ')
            
                                 
            if len(K_words) > len(k_words):
                # assumption: We can not have more common words than the word count of the smallest job-title
                min_count = len(k_words) if min_count > len(k_words) else min_count
                for word in K_words:
                    if word in k_words:
                        sim.append(word)
            else:
                # assumption: We can not have more common words than the word count of the smallest job-title
                min_count = len(K_words) if min_count > len(K_words) else min_count
                for word in k_words:
                    if word in K_words:
                        sim.append(word)
            if len(sim) >= min_count:
                print('##################################################')
                print(sim)
                if k not in similar.keys():
                    similar[k] = []
                    similar[k].append(K)
                else:
                    similar[k].append(K)
            else: #if the job title doesn't fall under the 52 work roles
                others.append(K)
            
# Exporting dictionary as a json file for use during sentence vectorization
data = similar
with open('similar.json', 'w') as f:
    json.dump(data, f)       

###############################################
WR:All Source-Collection Manager
WR:All Source-Collection Requirements Manager
WR:All-Source Analyst
WR:Authorizing Official/Designating Representative
WR:Communications Security (COMSEC) Manager
WR:Cyber Crime Investigator
WR:Cyber Defense Analyst
WR:Cyber Defense Forensics Analyst
WR:Cyber Defense Incident Responder
WR:Cyber Defense Infrastructure Support Specialist
WR:Cyber Instructional Curriculum Developer
WR:Cyber Instructor
WR:Cyber Intel Planner
WR:Cyber Legal Advisor
WR:Cyber Operator
WR:Cyber Ops Planner
WR:Cyber Policy and Strategy Planner
WR:Cyber Workforce Developer and Manager
WR:Data Analyst
WR:Database Administrator
WR:Enterprise Architect
WR:Executive Cyber Leadership
WR:Exploitation Analyst
WR:Information Systems Security Developer
WR:Information Systems Security Manager
WR:IT Investment/Portfolio Manager
WR:IT Program Auditor
WR:IT Project Manager
WR:Knowledge Manager
WR:Law Enforcement/Counterintelligence Forensics Ana

WR:Cyber Workforce Developer and Manager
WR:Data Analyst
WR:Database Administrator
WR:Enterprise Architect
WR:Executive Cyber Leadership
WR:Exploitation Analyst
WR:Information Systems Security Developer
WR:Information Systems Security Manager
WR:IT Investment/Portfolio Manager
WR:IT Program Auditor
WR:IT Project Manager
WR:Knowledge Manager
WR:Law Enforcement/Counterintelligence Forensics Analyst
WR:Mission Assessment Specialist
WR:Multi-Disciplined Language Analyst
WR:Network Operations Specialist
WR:Partner Integration Planner
WR:Privacy Officer/Privacy Compliance Manager
WR:Product Support Manager
WR:Program Manager
WR:Research & Development Specialist
WR:Secure Software Assessor
WR:Security Architect
WR:Security Control Assessor
WR:Software Developer
WR:System Administrator
WR:System Testing and Evaluation Specialist
WR:Systems Developer
WR:Systems Requirements Planner
WR:Systems Security Analyst
WR:Target Developer
WR:Target Network Analyst
WR:Technical Support Specialist
WR:Threa

['information', 'security']
WR:IT Investment/Portfolio Manager
WR:IT Program Auditor
WR:IT Project Manager
WR:Knowledge Manager
WR:Law Enforcement/Counterintelligence Forensics Analyst
WR:Mission Assessment Specialist
WR:Multi-Disciplined Language Analyst
WR:Network Operations Specialist
WR:Partner Integration Planner
WR:Privacy Officer/Privacy Compliance Manager
WR:Product Support Manager
WR:Program Manager
WR:Research & Development Specialist
WR:Secure Software Assessor
WR:Security Architect
WR:Security Control Assessor
WR:Software Developer
WR:System Administrator
WR:System Testing and Evaluation Specialist
WR:Systems Developer
WR:Systems Requirements Planner
WR:Systems Security Analyst
WR:Target Developer
WR:Target Network Analyst
WR:Technical Support Specialist
WR:Threat/Warning Analyst
WR:Vulnerability Assessment Analyst
###############################################
WR:All Source-Collection Manager
WR:All Source-Collection Requirements Manager
WR:All-Source Analyst
WR:Authorizi

WR:Cyber Defense Incident Responder
WR:Cyber Defense Infrastructure Support Specialist
WR:Cyber Instructional Curriculum Developer
WR:Cyber Instructor
WR:Cyber Intel Planner
WR:Cyber Legal Advisor
WR:Cyber Operator
WR:Cyber Ops Planner
WR:Cyber Policy and Strategy Planner
WR:Cyber Workforce Developer and Manager
WR:Data Analyst
WR:Database Administrator
WR:Enterprise Architect
WR:Executive Cyber Leadership
WR:Exploitation Analyst
WR:Information Systems Security Developer
WR:Information Systems Security Manager
WR:IT Investment/Portfolio Manager
WR:IT Program Auditor
WR:IT Project Manager
WR:Knowledge Manager
WR:Law Enforcement/Counterintelligence Forensics Analyst
WR:Mission Assessment Specialist
WR:Multi-Disciplined Language Analyst
WR:Network Operations Specialist
WR:Partner Integration Planner
WR:Privacy Officer/Privacy Compliance Manager
WR:Product Support Manager
WR:Program Manager
WR:Research & Development Specialist
WR:Secure Software Assessor
WR:Security Architect
WR:Security C

WR:Database Administrator
WR:Enterprise Architect
WR:Executive Cyber Leadership
WR:Exploitation Analyst
WR:Information Systems Security Developer
WR:Information Systems Security Manager
WR:IT Investment/Portfolio Manager
WR:IT Program Auditor
WR:IT Project Manager
WR:Knowledge Manager
WR:Law Enforcement/Counterintelligence Forensics Analyst
WR:Mission Assessment Specialist
WR:Multi-Disciplined Language Analyst
WR:Network Operations Specialist
WR:Partner Integration Planner
WR:Privacy Officer/Privacy Compliance Manager
WR:Product Support Manager
WR:Program Manager
WR:Research & Development Specialist
WR:Secure Software Assessor
WR:Security Architect
WR:Security Control Assessor
WR:Software Developer
WR:System Administrator
WR:System Testing and Evaluation Specialist
WR:Systems Developer
WR:Systems Requirements Planner
WR:Systems Security Analyst
WR:Target Developer
WR:Target Network Analyst
WR:Technical Support Specialist
WR:Threat/Warning Analyst
WR:Vulnerability Assessment Analyst
###

...

['cyber']
WR:Cyber Defense Incident Responder
##################################################
['cyber']
WR:Cyber Defense Infrastructure Support Specialist
##################################################
['cyber']
WR:Cyber Instructional Curriculum Developer
##################################################
['cyber']
WR:Cyber Instructor
##################################################
['cyber']
WR:Cyber Intel Planner
##################################################
['cyber']
WR:Cyber Legal Advisor
##################################################
['cyber']
WR:Cyber Operator
##################################################
['cyber']
WR:Cyber Ops Planner
##################################################
['cyber']
WR:Cyber Policy and Strategy Planner
##################################################
['cyber', 'and']
WR:Cyber Workforce Developer and Manager
##################################################
['cyber', 'and']
WR:Data Analyst
WR:Database Administrator
WR:Enterpr

WR:All Source-Collection Requirements Manager
WR:All-Source Analyst
##################################################
['analyst']
WR:Authorizing Official/Designating Representative
WR:Communications Security (COMSEC) Manager
WR:Cyber Crime Investigator
##################################################
['cyber']
WR:Cyber Defense Analyst
##################################################
['cyber', 'analyst']
WR:Cyber Defense Forensics Analyst
##################################################
['cyber', 'analyst']
WR:Cyber Defense Incident Responder
##################################################
['cyber']
WR:Cyber Defense Infrastructure Support Specialist
##################################################
['cyber']
WR:Cyber Instructional Curriculum Developer
##################################################
['cyber']
WR:Cyber Instructor
##################################################
['cyber']
WR:Cyber Intel Planner
##################################################
['cyber']
WR

##################################################
['security']
WR:Software Developer
WR:System Administrator
##################################################
['system']
WR:System Testing and Evaluation Specialist
##################################################
['system']
WR:Systems Developer
WR:Systems Requirements Planner
WR:Systems Security Analyst
##################################################
['security']
WR:Target Developer
WR:Target Network Analyst
WR:Technical Support Specialist
WR:Threat/Warning Analyst
WR:Vulnerability Assessment Analyst
###############################################
WR:All Source-Collection Manager
WR:All Source-Collection Requirements Manager
WR:All-Source Analyst
##################################################
['analyst']
WR:Authorizing Official/Designating Representative
WR:Communications Security (COMSEC) Manager
##################################################
['security']
WR:Cyber Crime Investigator
WR:Cyber Defense Analyst
#############

WR:Executive Cyber Leadership
##################################################
['cyber']
WR:Exploitation Analyst
##################################################
['analyst']
WR:Information Systems Security Developer
WR:Information Systems Security Manager
WR:IT Investment/Portfolio Manager
WR:IT Program Auditor
WR:IT Project Manager
WR:Knowledge Manager
WR:Law Enforcement/Counterintelligence Forensics Analyst
##################################################
['analyst']
WR:Mission Assessment Specialist
WR:Multi-Disciplined Language Analyst
##################################################
['analyst']
WR:Network Operations Specialist
WR:Partner Integration Planner
WR:Privacy Officer/Privacy Compliance Manager
WR:Product Support Manager
WR:Program Manager
WR:Research & Development Specialist
WR:Secure Software Assessor
WR:Security Architect
WR:Security Control Assessor
WR:Software Developer
WR:System Administrator
WR:System Testing and Evaluation Specialist
WR:Systems Developer
WR:

##################################################
['analyst']
WR:Authorizing Official/Designating Representative
WR:Communications Security (COMSEC) Manager
WR:Cyber Crime Investigator
WR:Cyber Defense Analyst
##################################################
['analyst']
WR:Cyber Defense Forensics Analyst
##################################################
['analyst']
WR:Cyber Defense Incident Responder
WR:Cyber Defense Infrastructure Support Specialist
WR:Cyber Instructional Curriculum Developer
WR:Cyber Instructor
WR:Cyber Intel Planner
WR:Cyber Legal Advisor
WR:Cyber Operator
WR:Cyber Ops Planner
WR:Cyber Policy and Strategy Planner
WR:Cyber Workforce Developer and Manager
WR:Data Analyst
##################################################
['analyst']
WR:Database Administrator
WR:Enterprise Architect
WR:Executive Cyber Leadership
WR:Exploitation Analyst
##################################################
['analyst']
WR:Information Systems Security Developer
WR:Information Systems Sec

In [118]:
others


['JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR PENETRATION TESTER',
 'JUNIOR P

In [126]:
len(similar.keys())


45

In [123]:
for k,v in similar.items():
    print(f'{k}: \n{v}')
    print('###########################################################')

Systems Security Analyst: 
['Info Security Analyst Senior/Penetration Tester - Senior (10...', 'INFORMATION SECURITY SPECIALIST (INTERNATIONAL SECURITY AFFA...', 'Detection Team - Computer Security Systems Specialist', 'Cyber Threat Team - Computer Security Systems Specialist', 'Information Systems Security Specialist (REMOTE)', 'Jr. Computer Security Systems Specialist (Cyber)', 'COMPUTER SECURITY SYSTEMS SPECIALIST', 'Staff Security Specialist, Security Architecture & Engineeri...', 'Computer Security Systems Specialist', 'COMPUTER SECURITY SYSTEMS SPECIALIST (SERVICE DESK)', 'Cyber Security Analyst - Expert', 'Cyber Threat Analyst/ Computer Security Systems Specialist,...', 'Senior Cyber Security Analyst', 'Cyber Security Analyst', 'Sr. Cyber Security Analyst', 'Cyber Security Analyst Associate SOC Tier 1 ITC#3073', 'Info Security Analyst - Remote', 'Junior Cyber Security Assurance Analyst', 'Security Awareness Lead - Cyber Security', 'Security Analyst', 'Junior Information Security

In [166]:
similar['System Testing and Evaluation Specialist']

KeyError: 'System Testing and Evaluation Specialist'

In [121]:
# Next Steps:
# 1. Lump KSAs of similar jobs together and save as a dictionary, where the keys are either Work-Role/WorkRole-ID  
# 2. Using cosine similarity, compare the KSAs of Work-Role/WorkRole-ID in the new dictionary with the KSAs of the same 
#    Work-Role/WorkRole-ID in syllabi
#
# NEXT, WORKING WITH CERTS AND JOB-POSTINGS....
# 
